In [1]:
import os
base_url = os.getenv('url')
base_url 

'http://111.223.37.52/v1'

In [2]:
# 1 Data Loading
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
import pandas as pd

# สร้างโมเดลที่ใช้เชื่อมต่อกับเซิร์ฟเวอร์กลาง
llm = ChatOpenAI(
    model='gpt-4o-mini',
    base_url=url,  # URL ของเซิร์ฟเวอร์กลาง
    api_key=api_key,  # API Key สำหรับการเข้าถึงเซิร์ฟเวอร์
    max_tokens=1000  # จำกัดจำนวนโทเค็นในคำตอบ
)

# โหลดข้อมูล
df = pd.read_csv("splitData.csv")  # โหลดไฟล์ข้อมูล


In [3]:
# 2 Distance Calculation : คำนวณระยะทางระหว่างตำแหน่งผู้ใช้กับสถานที่ใน DataFrame
from geopy.distance import geodesic

def calculate_distance(user_location, place_location):
    return geodesic(user_location, place_location).kilometers

# ลบแถวที่มีค่า NaN ในคอลัมน์ LATITUDE_LOCATION และ LONGITUDE_LOCATION
df = df.dropna(subset=['LATITUDE_LOCATION', 'LONGITUDE_LOCATION'])


In [4]:
# 3  Keyword Extraction : ใช้ LLM เพื่อแยกคำสำคัญจากคำถามของผู้ใช้
def extract_keywords_from_query(query: str) -> str:
    """
    ใช้โมเดล LLM เพื่อแยกคำสำคัญจากคำถามของผู้ใช้
    """
    # ส่งคำถามไปยังโมเดล LLM
    response = llm.invoke(f"แยกคำสำคัญจากคำถามนี้: '{query}' และแสดงคำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้")
    
    # ตรวจสอบผลลัพธ์และคืนค่าคำสำคัญ
    return response.content.strip()


In [5]:
# 4 Data Filtering : กรองข้อมูลใน DataFrame ตามคำสำคัญ เช่น จังหวัด, ประเภท, ระยะทาง ฯลฯ 
# คำนวณระยะทางแล้วเก็บไว้ในคอลัมน์ DISTANCE และกรองข้อมูลจากระยะทางที่คำนวณได้
def filter_data(df: pd.DataFrame, thai_name: str = None, region: str = None, subdistrict: str = None,
                district: str = None, province: str = None, category: str = None, subtype: str = None,
                user_location: tuple = None, radius: float = None) -> pd.DataFrame:
    """
    ฟังก์ชันกรองข้อมูลจาก DataFrame ตามค่าที่ระบุในคอลัมน์ต่างๆ และพิกัดระยะทาง
    """
    if df.empty:
        print("DataFrame is empty!")
        return pd.DataFrame()

    print("Initial DataFrame shape:", df.shape)

    # กรองข้อมูลตามคอลัมน์
    if thai_name:
        df = df[df['ATT_NAME_TH'].str.contains(thai_name, na=False, case=False)]
    if region:
        df = df[df['REGION_NAME_TH'].str.contains(region, na=False, case=False)]
    if subdistrict:
        df = df[df['SUBDISTRICT_NAME_TH'].str.contains(subdistrict, na=False, case=False)]
    if district:
        df = df[df['DISTRICT_NAME_TH'].str.contains(district, na=False, case=False)]
    if province:
        df = df[df['PROVINCE_NAME_TH'].str.contains(province, na=False, case=False)]
    if category:
        df = df[df['ATTR_CATAGORY_TH'].str.contains(category, na=False, case=False)]
    if subtype:
        df = df[df['ATTR_SUB_TYPE_TH'].str.contains(subtype, na=False, case=False)]

    # กรองข้อมูลตามระยะทาง (ถ้ากำหนดพิกัดผู้ใช้และรัศมี)
    if user_location and radius:
        df['DISTANCE'] = df.apply(
            lambda row: calculate_distance(user_location, (row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION'])), axis=1
        )
        df = df[df['DISTANCE'] <= radius]

    print("Filtered DataFrame shape:", df.shape)
    return df


In [6]:
# 5 Tool Creation : ให้ LLM สามารถเรียกใช้ฟังก์ชัน filter_data ได้ (Function Calling)
from langchain.agents import Tool

# สร้าง Custom Tool
filter_data_tool = Tool(
    name="Filter Data",
    func=filter_data,
    description="กรองข้อมูลจาก DataFrame ตามคอลัมน์ต่างๆ เช่น ATT_NAME_TH, REGION_NAME_TH รวมถึงการกรองระยะทางจากพิกัดผู้ใช้"
)


In [7]:
# 6 Agent Creation : ใช้ create_pandas_dataframe_agent รวม LLM กับ DataFrame และ Tool
# ตั้งค่าให้ Agent สามารถทำงานแบบ Function Calling ได้
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True,  # ใช้ตัวเลือกนี้อย่างระมัดระวัง
    agent_type="tool-calling",
    tools=[filter_data_tool]
)


c:\llmproject\testenv\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'tools': [Tool(name='Filter Data', description='กรองข้อมูลจาก DataFrame ตามคอลัมน์ต่างๆ เช่น ATT_NAME_TH, REGION_NAME_TH รวมถึงการกรองระยะทางจากพิกัดผู้ใช้', func=<function filter_data at 0x000002042777DA80>)]} which are no longer supported.
  warnings.warn(


In [11]:
# 7 User Query Processing : รับคำถามของผู้ใช้ & แยกคำสำคัญ & เพื่อกรองข้อมูลตามคำสำคัญ และ ระยะทาง
# ตัวอย่างคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# ขั้นตอนที่ 1: กรองข้อมูลตามระยะทางจากตำแหน่งของผู้ใช้ก่อน
filtered_by_location = filter_data(
    df=df, 
    user_location=user_location,  # กรองข้อมูลตามพิกัดของผู้ใช้
    radius=radius  # กรองข้อมูลที่อยู่ในรัศมี
)

# ขั้นตอนที่ 2: กรองข้อมูลตามคำสำคัญจากคำถามของผู้ใช้
province = None
category = None

# ตัวอย่างการตรวจสอบคำสำคัญและกรองข้อมูล
if 'จังหวัด' in keywords:
    province = [word for word in keywords.split() if 'จังหวัด' in word][0]  # ดึงจังหวัดจากคำสำคัญ
if 'แหล่งท่องเที่ยว' in keywords:
    category = [word for word in keywords.split() if 'แหล่งท่องเที่ยว' in word][0]  # ดึงประเภทจากคำสำคัญ

# ขั้นตอนที่ 3: กรองข้อมูลตามคำสำคัญที่ได้จากคำถามหลังจากที่กรองระยะทางแล้ว
filtered_by_keywords = filter_data(
    df=filtered_by_location,  # ใช้ข้อมูลที่กรองตามระยะทาง
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตรและตามคำสำคัญ:")
print(filtered_by_keywords[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัด
4. นครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- นครปฐม
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สถานที่ธรรมชาติ
- พื้นที่สีเขียว
- อุทยาน
- น้ำตก
- สวนสาธารณะ

สามารถใช้คำสำคัญเหล่านี้ในการค้นหาข้อมูลเพิ่มเติมเกี่ยวกับแหล่งท่องเที่ยวในจังหวัดนครปฐมได้ค่ะ
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)
Initial DataFrame shape: (330, 16)
Filtered DataFrame shape: (0, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตรและตามคำสำคัญ:
Empty DataFrame
Columns: [ATT_NAME_TH, PROVINCE_NAME_TH, LATITUDE_LOCATION, LONGITUDE_LOCATION]
Index: []


In [10]:
# 8 แสดงแผนที่และข้อมูลชื่อสถานที่ที่กรองได้ และ แสดงตำแหน่งของuser

import ipywidgets as widgets
import folium
from IPython.display import display

# ฟังก์ชันสร้างแผนที่พร้อมแสดงหมุดของผู้ใช้
def create_map(center_lat, center_lon, zoom_start=12, user_lat=None, user_lon=None):
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start)
    
    # หากมีข้อมูลตำแหน่งของผู้ใช้ ให้แสดงหมุดของผู้ใช้
    if user_lat and user_lon:
        folium.Marker([user_lat, user_lon],
                      popup="คุณอยู่ที่นี่",
                      icon=folium.Icon(color="blue", icon="info-sign")).add_to(m)  # หมุดสีน้ำเงิน
    
    return m

# ฟังก์ชันอัปเดตแผนที่เมื่อเลือกสถานที่
def update_map(change):
    # ค้นหาสถานที่ที่ถูกเลือกใน Dropdown
    selected_place = filtered_by_location[filtered_by_location['ATT_NAME_TH'] == dropdown.value].iloc[0]
    lat, lon = selected_place['LATITUDE_LOCATION'], selected_place['LONGITUDE_LOCATION']
    
    # ใช้ตำแหน่งผู้ใช้จาก user_location ที่ได้รับจากโค้ดก่อนหน้า
    user_lat, user_lon = user_location   # พิกัดของผู้ใช้
    
    m = create_map(lat, lon, user_lat=user_lat, user_lon=user_lon)
    
    # แสดงหมุดของสถานที่ที่เลือก
    for _, row in filtered_by_location.iterrows():
        folium.Marker([row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION']],
                      popup=row['ATT_NAME_TH'],
                      icon=folium.Icon(color="green")).add_to(m)
    display(m)

# สร้าง Dropdown สำหรับเลือกสถานที่ที่กรองแล้ว
dropdown = widgets.Dropdown(
    options=filtered_by_location['ATT_NAME_TH'].tolist(),  # ใช้สถานที่ที่กรองแล้ว
    description='สถานที่:',
    value=filtered_by_location['ATT_NAME_TH'].iloc[0]  # กำหนดค่าเริ่มต้น
)

# ตั้งค่าการอัปเดตเมื่อเปลี่ยนค่า Dropdown
dropdown.observe(update_map, names='value')

# สร้าง Layout สำหรับการแสดงผล
display(widgets.VBox([dropdown]))

# แสดงแผนที่เริ่มต้น
initial_place = filtered_by_location.iloc[0]  # ใช้สถานที่ที่กรองแล้วเป็นจุดเริ่มต้น
# ใช้ตำแหน่งผู้ใช้จาก user_location ที่ได้รับจากโค้ดก่อนหน้า
user_lat, user_lon = 14.022788, 99.978337  # พิกัดของผู้ใช้

initial_map = create_map(initial_place['LATITUDE_LOCATION'], initial_place['LONGITUDE_LOCATION'], user_lat=user_lat, user_lon=user_lon)
# แสดงหมุดของสถานที่
for _, row in filtered_by_location.iterrows():
    folium.Marker([row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION']],
                  popup=row['ATT_NAME_TH'],
                  icon=folium.Icon(color="green")).add_to(initial_map)
display(initial_map)


In [14]:
def filter_data_by_keywords(df, keywords):
    # กรองข้อมูลตามคำสำคัญที่ได้จากคำถาม
    # ตัวอย่างการกรองตามคำสำคัญ
    categories = ["น้ำตก", "ศูนย์ศึกษาธรรมชาติ", "อุทยาน", "สวนสาธารณะ"]
    
    filtered_data = df.copy()  # ทำสำเนาของ DataFrame
    for keyword in keywords:
        # กรองข้อมูลตามคำสำคัญ
        filtered_data = filtered_data[filtered_data["ATTR_SUB_TYPE_TH"].str.contains(keyword, case=False, na=False)]
    
    return filtered_data


In [16]:
# 1. รับคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# 2. ใช้ฟังก์ชันแยกคำสำคัญจากคำถาม
keywords = extract_keywords_from_query(user_query)

# 3. กรองข้อมูลตามระยะทาง
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 20  # กำหนดรัศมีในการค้นหาข้อมูล

# ฟังก์ชันกรองข้อมูลตามระยะทาง
filtered_by_location = filter_data(
    df=df, 
    user_location=user_location,  
    radius=radius
)

# 4. กรองข้อมูลตามคำสำคัญที่ได้จากคำถาม (ตัวอย่าง)
# เราจะใช้คำสำคัญที่ได้มาเพื่อค้นหาในประเภท ATTR_SUB_TYPE_TH หรือข้อมูลที่เกี่ยวข้อง
filtered_by_keywords = filter_data_by_keywords(
    df=filtered_by_location,  # ข้อมูลที่กรองตามระยะทาง
    keywords=keywords  # คำสำคัญที่ได้จากคำถาม
)

# 5. แสดงผลลัพธ์
print("สถานที่ท่องเที่ยวที่กรองแล้วตามคำสำคัญและระยะทาง:")
print(filtered_by_keywords[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (22, 16)
สถานที่ท่องเที่ยวที่กรองแล้วตามคำสำคัญและระยะทาง:
Empty DataFrame
Columns: [ATT_NAME_TH, PROVINCE_NAME_TH, LATITUDE_LOCATION, LONGITUDE_LOCATION]
Index: []


In [20]:
print(filtered_by_location)


                                            ATT_NAME_TH  \
563                                   เมืองเก่ากำแพงแสน   
573                                     หมู่บ้านไทยโซ่ง   
1071                               ตลาดบางหลวง ร.ศ. 122   
2391                                       วัดไร่แตงทอง   
2587                                       วัดทับกระดาน   
3383                        สวนป่าสมุนไพร วัดปลักไม้ลาย   
3841                          อุทยานแมลงเฉลิมพระเกียรติ   
4621  สวนสาธารณะเฉลิมพระเกียรติ 80 พรรษา พระบาทสมเด็...   
4756                                วัดประชาราษฎร์บำรุง   
4757                                      วัดปลักไม้ลาย   
4806                                     วัดอรัญญิการาม   
4807                                         วัดอ้อน้อย   
4879                             โรงเรียนการบินกำแพงแสน   
4942                            โรงเจจี่เต็กลิ้ม นครปฐม   
5122                      สวนกล้วยไม้ สุวรรณภูมิออร์คิด   
5259                                   ชุมชนบ้านเกาะแรด 

# ตรวจสอบระบบว่าทำมัยถึงกรองคำสำคัญหลังจากการกรองระยะทางไม่ได้

In [12]:
print(filtered_by_location.head())
print(filtered_by_location.shape)


                      ATT_NAME_TH                 ATT_NAME_EN  \
48             ชุมชนเกาะลัดอีแท่น                         NaN   
130   พิพิธภัณฑสถานแห่งชาติอู่ทอง    Au Thong National Museum   
165        วัดราษฎร์ศรัทธากะยาราม        Wat Ratsatthakayaram   
191  กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย                      Banana   
203              วูดแลนด์เมืองไม้  Woodland Museum and Resort   

                                   ATT_NEARBY_LOCATION  ATT_ADDRESS  \
48   ขับรถขึ้นสะพานมงคลรัฐประชานูกุล ข้ามแม่น้ำท่าจ...          NaN   
130     ตั้งอยู่ใกล้ที่ว่าการอำเภออู่ทอง บนถนนมาลัยแมน         1707   
165              อยู่หมู่ 2 ตำบลหลักสาม อำเภอบ้านแพ้ว        หมู่ 2   
191                                            ทุ่งสมอ     22 หมู่3   
203  ตั้งอยู่ที่ตำบลดอนแฝก ห่างจากตลาดท่านาไปทางทิศ...  15/1 หมู่ 4   

    REGION_NAME_TH SUBDISTRICT_NAME_TH DISTRICT_NAME_TH PROVINCE_NAME_TH  \
48         ภาคกลาง              บางเตย          สามพราน           นครปฐม   
130        ภาคกลาง            

In [13]:
filtered_by_keywords = filtered_by_location[filtered_by_location['ATTR_CATAGORY_TH'].str.contains("แหล่งท่องเที่ยวทางธรรมชาติ", na=False)]
print(filtered_by_keywords)


                               ATT_NAME_TH                       ATT_NAME_EN  \
927                            เขาช่องพราน                   Khao Chong Pran   
1184                 ถ้ำค้างคาวเขาช่องพราน     Bat Cave at Khao Chong Phran    
1592                                 ภูค้อ                           Phu Koh   
2175                       ต้นจามจุรียักษ์            Giant  Monkey Pod Tree   
2538                         เขื่อนแม่กลอง                     Mae Klong Dam   
3213                        วนอุทยานพุม่วง             Phu Muang Forest Park   
3845  อุทยานปลาวัดห้วยพลูในแม่น้ำนครชัยศรี     Fish Sanctuary Wat Huai Phlu    
3913                                ภูลมโล                        Phu Lom Lo   
4450                         หาดทรายท่าล้อ                     Tha Lor Beach   
4719                          นาบัวลุงแจ่ม                             ไม่มี   
5914                อุทยานมัจฉาวังสังกะวาด                Sangawat Fish Park   
6259                        คลองมหาสวัสด

In [11]:
# 7 User Query Processing : รับคำถามของผู้ใช้ & แยกคำสำคัญ & เพื่อกรองข้อมูลตามคำสำคัญ และ ระยะทาง
# รับค่ามาจากinput user ที่กรองแล้ว
# ตัวอย่างคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# การกรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
# แยกคำสำคัญที่เกี่ยวข้องกับการกรองข้อมูลออก
province = None
category = None

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามพิกัดหลังจากกรองคำสั่งผู้ใช้แล้ว
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# กรองข้อมูลตามคำสำคัญของผู้ใช้
filtered_by_user_request = filter_data(
    df=filtered_by_location, 
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)


# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:")
print(filtered_by_location[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย" ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่สามารถใช้กรองข้อมูลได้ ได้แก่:

- ท่องเที่ยว
- ธรรมชาติ
- จังหวัด
- นครปฐม

คำเหล่านี้สามารถใช้ในการค้นหาข้อมูลเกี่ยวกับแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพ
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)
Initial DataFrame shape: (330, 16)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                       ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
48              ชุมชนเกาะลัดอีแท่น           นครปฐม          13.756728   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง       สุพรรณบุรี          14.372874   
165         วัดราษฎร์ศรัทธากะยาราม        สมุทรสาคร          13.590556   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย        กาญจนบุรี          14.071364   
203               วูดแลนด์เมืองไม้           นครปฐม          13.858140   
...          

In [10]:
# 7 User Query Processing : รับคำถามของผู้ใช้ & แยกคำสำคัญ & เพื่อกรองข้อมูลตามคำสำคัญ และ ระยะทาง
# รับค่ามาจากinput user ที่กรองแล้ว
# ตัวอย่างคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# การกรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
# แยกคำสำคัญที่เกี่ยวข้องกับการกรองข้อมูลออก
province = None
category = None

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามพิกัดหลังจากกรองคำสั่งผู้ใช้แล้ว
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# กรองข้อมูลตามคำสำคัญของผู้ใช้
filtered_by_user_request = filter_data(
    df=filtered_by_location, 
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)


# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย" ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัด
4. นครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- จังหวัดนครปฐม
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สถานที่ธรรมชาติ
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)
Initial DataFrame shape: (330, 16)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                       ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
48              ชุมชนเกาะลัดอีแท่น           นครปฐม          13.756728   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง       สุพรรณบุรี          14.372874   
165         วัดราษฎร์ศรัทธากะยาราม        สมุทรสาคร          13.590556   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย        กาญจนบุรี          14.071364   
203               วูดแลนด์เมืองไม้           นครปฐม          13.858140   
...                            ...              ...       

In [9]:
# 7 User Query Processing : รับคำถามของผู้ใช้ & แยกคำสำคัญ & เพื่อกรองข้อมูลตามคำสำคัญ และ ระยะทาง
# รับค่ามาจากinput user ที่กรองแล้ว
# ตัวอย่างคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# การกรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
# แยกคำสำคัญที่เกี่ยวข้องกับการกรองข้อมูลออก
province = None
category = None

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามพิกัดหลังจากกรองคำสั่งผู้ใช้แล้ว
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# กรองข้อมูลตามคำสำคัญของผู้ใช้
filtered_by_user_request = filter_data(
    df=filtered_by_location, 
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)

common_rows = filtered_by_location[
    filtered_by_location[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']]
    .isin(filtered_by_user_request[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])
    .all(axis=1)
]

if common_rows.empty:
    print("ไม่พบแถวที่เหมือนกันระหว่าง DataFrame")
else:
    print("แถวที่เหมือนกันระหว่าง DataFrame:")
    print(common_rows[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- จังหวัด
- นครปฐม
- สถานที่ท่องเที่ยว
- การเดินทาง
- กิจกรรมกลางแจ้ง

สามารถใช้คำเหล่านี้ในการค้นหาข้อมูลเพิ่มเติมเกี่ยวกับแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้ค่ะ
Initial DataFrame shape: (8241, 15)
Filtered DataFrame shape: (330, 16)
Initial DataFrame shape: (330, 16)
Filtered DataFrame shape: (330, 16)
แถวที่เหมือนกันระหว่าง DataFrame:
                       ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
48              ชุมชนเกาะลัดอีแท่น           นครปฐม          13.756728   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง       สุพรรณบุรี          14.372874   
165         วัดราษฎร์ศรัทธากะยาราม        สมุทรสาคร          13.590556   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย        กาญจนบุรี          14.071364   
203               วูดแลนด์เมืองไม้           นครปฐม

In [18]:
# 7 User Query Processing : รับคำถามของผู้ใช้ & แยกคำสำคัญ & เพื่อกรองข้อมูลตามคำสำคัญ และ ระยะทาง
# รับค่ามาจาก input user ที่กรองแล้ว
# ตัวอย่างคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# การกรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
# แยกคำสำคัญที่เกี่ยวข้องกับการกรองข้อมูลออก
province = None
category = None

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามพิกัดของผู้ใช้ก่อน
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# เก็บ DataFrame ที่กรองแล้วจาก filtered_by_location
filtered_df_location = filtered_by_location.copy()

# กรองข้อมูลตามคำสำคัญของผู้ใช้ จาก DataFrame ที่กรองตามพิกัดแล้ว
filtered_by_user_request = filter_data(
    df=filtered_df_location, 
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)

print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี" ,radius, "กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย" ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ประเภทของแหล่งท่องเที่ยว (ธรรมชาติ, วัฒนธรรม, ประวัติศาสตร์ ฯลฯ)
- สถานที่ (นครปฐม)
- กิจกรรมที่ทำได้ (เดินป่า, ดูนก, พักผ่อน ฯลฯ)
- ความนิยม (แหล่งท่องเที่ยวยอดนิยม, แหล่งท่องเที่ยวที่แนะนำ ฯลฯ)
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)
Initial DataFrame shape: (330, 16)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                       ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
48              ชุมชนเกาะลัดอีแท่น           นครปฐม          13.756728   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง       สุพรรณบุรี          14.372874   
165         วัดราษฎร์ศรัทธากะยาราม        สมุทรสาคร          13.590556   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย        กาญจนบุรี          14.071364   
203               วูดแลนด์เมืองไม้

In [9]:
# ถ้าผลลัพธ์จาก filtered_by_location มีจังหวัดที่เกี่ยวข้อง
# ให้ใช้จังหวัดจากผลลัพธ์นี้เป็นค่า province
province = filtered_df_location['PROVINCE_NAME_TH'].iloc[0] if not filtered_df_location.empty else None

# ตรวจสอบว่ามีค่าในคอลัมน์ ATTR_CATAGORY_TH หรือไม่
if not filtered_df_location['ATTR_CATAGORY_TH'].isna().all():
    # เลือกค่าหมวดหมู่แรกที่พบใน ATTR_CATAGORY_TH
    category = filtered_df_location['ATTR_CATAGORY_TH'].iloc[0]
else:
    category = None  # ถ้าไม่มีค่าใน ATTR_CATAGORY_TH ให้ตั้งเป็น None

# กรองข้อมูลตามคำสำคัญของผู้ใช้ จาก DataFrame ที่กรองตามพิกัดแล้ว
if category is None:
    # ถ้าไม่มี category ให้กรองเฉพาะจังหวัด
    filtered_by_user_request = filtered_df_location[filtered_df_location['PROVINCE_NAME_TH'] == province]
else:
    # กรองทั้งจังหวัดและหมวดหมู่
    filtered_by_user_request = filtered_df_location[
        (filtered_df_location['PROVINCE_NAME_TH'] == province) &
        (filtered_df_location['ATTR_CATAGORY_TH'] == category)
    ]

คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- นครปฐม
- แหล่งท่องเที่ยวธรรมชาติ
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สถานที่ทางธรรมชาติ

โดยการใช้คำสำคัญเหล่านี้สามารถช่วยในการค้นหาข้อมูลที่เกี่ยวข้องกับแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพมากขึ้นค่ะ
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                           ATT_NAME_TH PROVINCE_NAME_TH  \
48                  ชุมชนเกาะลัดอีแท่น           นครปฐม   
203                   วูดแลนด์เมืองไม้           นครปฐม   
282   พิพิธภัณฑ์โรงเรียนวัดไร่ขิงวิทยา           นครปฐม   
410                          ตลาดในสวน           นครปฐม   
563                  เมืองเก่ากำแพงแสน           นครปฐม   
...                                ...              ...   
81

In [8]:
# 7 User Query Processing : รับคำถามของผู้ใช้ & แยกคำสำคัญ & เพื่อกรองข้อมูลตามคำสำคัญ และ ระยะทาง

# ตัวอย่างคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดกรุงเทพให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# การกรองข้อมูลตามพิกัดของผู้ใช้ก่อน
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# เก็บ DataFrame ที่กรองแล้วจาก filtered_by_location
filtered_df_location = filtered_by_location.copy()

# กำหนดค่า province และ category จากข้อมูลที่กรองได้
province = filtered_df_location['PROVINCE_NAME_TH'].iloc[0] if not filtered_df_location.empty else None

# ตรวจสอบว่ามีค่าในคอลัมน์ ATTR_CATAGORY_TH หรือไม่
if not filtered_df_location['ATTR_CATAGORY_TH'].isna().all():
    # เลือกค่าหมวดหมู่แรกที่พบใน ATTR_CATAGORY_TH
    category = filtered_df_location['ATTR_CATAGORY_TH'].iloc[0]
else:
    category = None  # ถ้าไม่มีค่าใน ATTR_CATAGORY_TH ให้ตั้งเป็น None

# การกรองข้อมูลตาม province และ category
if province is not None:
    if category is None:
        # ถ้าไม่มี category ให้กรองเฉพาะจังหวัด
        filtered_by_user_request = filtered_df_location[
            filtered_df_location['PROVINCE_NAME_TH'] == province
        ]
    else:
        # กรองทั้งจังหวัดและหมวดหมู่
        filtered_by_user_request = filtered_df_location[
            (filtered_df_location['PROVINCE_NAME_TH'] == province) &
            (filtered_df_location['ATTR_CATAGORY_TH'] == category)
        ]
else:
    # ถ้าไม่มี province ข้อมูลจะไม่ถูกกรอง
    filtered_by_user_request = filtered_df_location

# แสดงผลลัพธ์
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี", radius, "กิโลเมตร:")
print(filtered_by_user_request[[
    'ATT_NAME_TH', 'PROVINCE_NAME_TH', 'ATTR_CATAGORY_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION'
]])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดกรุงเทพให้หน่อย" ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดกรุงเทพ

คำสำคัญที่เกี่ยวข้องสำหรับการกรองข้อมูล:

- ท่องเที่ยว
- ธรรมชาติ
- กรุงเทพ
- แหล่งท่องเที่ยวธรรมชาติ
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สวนสาธารณะ
- อุทยาน
- พื้นที่สีเขียว

คุณสามารถใช้คำสำคัญเหล่านี้ในการค้นหาข้อมูลหรือกรองข้อมูลเกี่ยวกับแหล่งท่องเที่ยวทางธรรมชาติในกรุงเทพได้ค่ะ
Initial DataFrame shape: (8241, 15)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                                            ATT_NAME_TH PROVINCE_NAME_TH  \
48                                   ชุมชนเกาะลัดอีแท่น           นครปฐม   
203                                    วูดแลนด์เมืองไม้           นครปฐม   
410                                           ตลาดในสวน           นครปฐม   
1864                                      ตลาดบ้านรังนก           นครปฐม   
2172                      สยามชัยหาดทรายขาว ทะเลนคร

In [18]:
# รับคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 15  

# กรองข้อมูลตามพิกัดก่อน
filtered_by_location = filter_data(
    df=df, 
    user_location=user_location,
    radius=radius
)

# หลังจากกรองตามระยะทางแล้ว กรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
province = None
category = None

# กรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
filtered_by_user_request = filter_data(
    df=filtered_by_location,  # ส่ง DataFrame ที่กรองตามระยะทางแล้ว
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี" ,radius, "กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'ATTR_CATAGORY_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัด
4. นครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- นครปฐม
- แหล่งท่องเที่ยวทางธรรมชาติ
- สถานที่ท่องเที่ยว
- กิจกรรมท่องเที่ยว
- จังหวัดนครปฐม

การใช้คำสำคัญเหล่านี้จะช่วยในการค้นหาข้อมูลที่เกี่ยวข้องกับแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพมากขึ้นค่ะ
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (15, 16)
Initial DataFrame shape: (15, 16)
Filtered DataFrame shape: (15, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 15 กิโลเมตร:
                                            ATT_NAME_TH  \
563                                   เมืองเก่ากำแพงแสน   
2391                                       วัดไร่แตงทอง   
2587                                       วัดทับกระดาน   
3383                        สวนป่าสมุนไพร วัดปลักไม้ลาย   
3841                          อุทยานแมลงเฉลิมพระเกียรติ

In [17]:
# Map Rendering
import ipywidgets as widgets
import folium
from IPython.display import display

# ฟังก์ชันสร้างแผนที่พร้อมแสดงหมุดของผู้ใช้
def create_map(center_lat, center_lon, zoom_start=12, user_lat=None, user_lon=None):
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start)
    
    # หากมีข้อมูลตำแหน่งของผู้ใช้ ให้แสดงหมุดของผู้ใช้
    if user_lat and user_lon:
        folium.Marker([user_lat, user_lon],
                      popup="คุณอยู่ที่นี่",
                      icon=folium.Icon(color="blue", icon="info-sign")).add_to(m)  # หมุดสีน้ำเงิน
    
    return m

# ฟังก์ชันอัปเดตแผนที่เมื่อเลือกสถานที่
def update_map(change):
    # ค้นหาสถานที่ที่ถูกเลือกใน Dropdown
    selected_place = filtered_by_location.head(10).reset_index(drop=True)
    selected_place = selected_place[selected_place['ATT_NAME_TH'] == dropdown.value].iloc[0]
    lat, lon = selected_place['LATITUDE_LOCATION'], selected_place['LONGITUDE_LOCATION']
    
    # ใช้ตำแหน่งผู้ใช้จาก user_location ที่ได้รับจากโค้ดก่อนหน้า
    user_lat, user_lon = user_location  # ใช้ตำแหน่งจากตัวแปร user_location
    
    m = create_map(lat, lon, user_lat=user_lat, user_lon=user_lon)
    
    # แสดงหมุดของสถานที่ที่เลือก
    for _, row in filtered_by_location.head(10).iterrows():  # ใช้ head(10) เพื่อแสดงแค่ 10 อันดับแรก
        folium.Marker([row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION']],
                      popup=row['ATT_NAME_TH'],
                      icon=folium.Icon(color="green")).add_to(m)
    display(m)

# สร้าง Dropdown สำหรับเลือกสถานที่ที่กรองแล้ว (จำกัด 10 อันดับแรก)
dropdown = widgets.Dropdown(
    options=filtered_by_location.head(10)['ATT_NAME_TH'].tolist(),  # ใช้สถานที่ที่กรองแล้ว 10 อันดับแรก
    description='สถานที่:',
    value=filtered_by_location.head(10)['ATT_NAME_TH'].iloc[0]  # กำหนดค่าเริ่มต้น
)

# ตั้งค่าการอัปเดตเมื่อเปลี่ยนค่า Dropdown
dropdown.observe(update_map, names='value')

# สร้าง Layout สำหรับการแสดงผล
display(widgets.VBox([dropdown]))

# แสดงแผนที่เริ่มต้น (แค่ 10 อันดับแรก)
initial_place = filtered_by_location.head(10).reset_index(drop=True).iloc[0]  # รีเซ็ตดัชนี
# ใช้ตำแหน่งผู้ใช้จาก user_location ที่ได้รับจากโค้ดก่อนหน้า
user_lat, user_lon = user_location  # ใช้ตำแหน่งจากตัวแปร user_location

initial_map = create_map(initial_place['LATITUDE_LOCATION'], initial_place['LONGITUDE_LOCATION'], user_lat=user_lat, user_lon=user_lon)
# แสดงหมุดของสถานที่
for _, row in filtered_by_location.head(10).iterrows():  # ใช้ head(10) เพื่อแสดงแค่ 10 อันดับแรก
    folium.Marker([row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION']],
                  popup=row['ATT_NAME_TH'],
                  icon=folium.Icon(color="green")).add_to(initial_map)
display(initial_map)


# ลองแก้ Data Filtering 

In [5]:
def filter_data(df: pd.DataFrame, thai_name: str = None, region: str = None, subdistrict: str = None,
                district: str = None, province: str = None, category: str = None, subtype: str = None,
                user_location: tuple = None, radius: float = None, filter_by_distance_first=False) -> pd.DataFrame:
    """
    ฟังก์ชันกรองข้อมูลจาก DataFrame ตามค่าที่ระบุในคอลัมน์ต่างๆ และพิกัดระยะทาง
    สามารถกำหนดลำดับการกรองได้ว่าให้กรองตามระยะทางก่อนหรือไม่
    """
    if df.empty:
        print("DataFrame is empty!")
        return pd.DataFrame()

    print("Initial DataFrame shape:", df.shape)

    # กรองข้อมูลตามระยะทาง (ถ้ากำหนดพิกัดผู้ใช้และรัศมี) - หากกำหนดให้กรองก่อน
    if filter_by_distance_first and user_location and radius:
        df['DISTANCE'] = df.apply(
            lambda row: calculate_distance(user_location, (row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION'])), axis=1
        )
        df = df[df['DISTANCE'] <= radius]
        print("Filtered by distance first:", df.shape)

    # กรองข้อมูลตามคำสำคัญที่ผู้ใช้ระบุ
    if thai_name:
        df = df[df['ATT_NAME_TH'].str.contains(thai_name, na=False, case=False)]
    if region:
        df = df[df['REGION_NAME_TH'].str.contains(region, na=False, case=False)]
    if subdistrict:
        df = df[df['SUBDISTRICT_NAME_TH'].str.contains(subdistrict, na=False, case=False)]
    if district:
        df = df[df['DISTRICT_NAME_TH'].str.contains(district, na=False, case=False)]
    if province:
        df = df[df['PROVINCE_NAME_TH'].str.contains(province, na=False, case=False)]
    if category:
        df = df[df['ATTR_CATAGORY_TH'].str.contains(category, na=False, case=False)]
    if subtype:
        df = df[df['ATTR_SUB_TYPE_TH'].str.contains(subtype, na=False, case=False)]

    # กรองข้อมูลตามระยะทาง (ถ้ากำหนดพิกัดผู้ใช้และรัศมี) - หากไม่กรองตามระยะทางก่อน
    if not filter_by_distance_first and user_location and radius:
        df['DISTANCE'] = df.apply(
            lambda row: calculate_distance(user_location, (row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION'])), axis=1
        )
        df = df[df['DISTANCE'] <= radius]

    print("Filtered DataFrame shape:", df.shape)
    return df


In [10]:
# รับคำถามของผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  

# กรองข้อมูลตามพิกัดก่อน
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius,
    filter_by_distance_first=True  # กรองตามระยะทางก่อน
)


# หลังจากกรองตามระยะทางแล้ว กรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
province = None
category = None

# กรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
filtered_by_user_request = filter_data(
    df=filtered_by_location,  # ส่ง DataFrame ที่กรองตามระยะทางแล้ว
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี" ,radius, "กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'ATTR_CATAGORY_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย" ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ประเภทท่องเที่ยว (เช่น ธรรมชาติ, วัฒนธรรม, ประวัติศาสตร์)
- จังหวัด (นครปฐม)
- กิจกรรม (เช่น เดินป่า, ดูนก, ถ่ายภาพ)
- สถานที่ (เช่น อุทยาน, น้ำตก, เขื่อน)
- ความนิยม (เช่น แหล่งท่องเที่ยวที่มีคนมาเยี่ยมชมมาก)

โดยการใช้คำสำคัญเหล่านี้สามารถช่วยในการค้นหาข้อมูลที่เกี่ยวข้องกับการท่องเที่ยวในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพมากขึ้น
Initial DataFrame shape: (8241, 15)
Filtered by distance first: (330, 16)
Filtered DataFrame shape: (330, 16)
Initial DataFrame shape: (330, 16)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                       ATT_NAME_TH  \
48              ชุมชนเกาะลัดอีแท่น   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง   
165         วัดราษฎร์ศรัทธากะยาราม   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย   
203               วูดแลนด์เมืองไม้   


In [13]:
# สมมุติว่า `keywords` จะต้องเป็น dictionary ที่เก็บคำสำคัญ
keywords = extract_keywords_from_query(user_query)

# ตรวจสอบว่า `keywords` เป็น dictionary หรือไม่
if isinstance(keywords, dict):
    province = keywords.get('province')
    category = keywords.get('category')
else:
    province = None
    category = None
    print("Error: 'keywords' is not a dictionary.")

# 1. กรองข้อมูลตามคำสำคัญที่ได้จากคำถามผู้ใช้ (กรองจังหวัดเป็นลำดับแรก)
filtered_by_user_request = filter_data(
    df=df,
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category   # กรองประเภทตามคำสำคัญที่ได้
)

# 2. กรองข้อมูลตามระยะทางหลังจากกรองจังหวัด
filtered_by_location = filter_data(
    df=filtered_by_user_request,
    user_location=user_location,  # พิกัดของผู้ใช้
    radius=radius,                # รัศมีที่กำหนด
    filter_by_distance_first=False  # กรองตามจังหวัดก่อนแล้วค่อยกรองตามระยะทาง
)

# 3. ตรวจสอบผลลัพธ์หลังจากกรองตามจังหวัดและระยะทาง
print(filtered_by_location[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


Error: 'keywords' is not a dictionary.
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (8241, 16)
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)
                       ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
48              ชุมชนเกาะลัดอีแท่น           นครปฐม          13.756728   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง       สุพรรณบุรี          14.372874   
165         วัดราษฎร์ศรัทธากะยาราม        สมุทรสาคร          13.590556   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย        กาญจนบุรี          14.071364   
203               วูดแลนด์เมืองไม้           นครปฐม          13.858140   
...                            ...              ...                ...   
8175              สมาภรณ์ การ์เด้น        สมุทรสาคร          13.603811   
8189                สวนส้มโอไทยทวี           นครปฐม          13.823086   
8196                 วัดญาณเวศกวัน           นครปฐม          13.770129   
8197                  วัดดอนยายหอม           นครปฐม          13.736541   
82

In [14]:
# สมมุติว่า `keywords` จะต้องเป็น dictionary ที่เก็บคำสำคัญ
keywords = extract_keywords_from_query(user_query)

# ตรวจสอบว่า `keywords` เป็น dictionary หรือไม่
if isinstance(keywords, dict):
    province = keywords.get('province')
    category = keywords.get('category')
else:
    province = None
    category = None
    print("Error: 'keywords' is not a dictionary.")

# 1. กรองข้อมูลตามคำสำคัญที่ได้จากคำถามผู้ใช้ (กรองจังหวัดเป็นลำดับแรก)
filtered_by_user_request = filter_data(
    df=df,
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category   # กรองประเภทตามคำสำคัญที่ได้
)

# 2. แสดงผลลัพธ์หลังจากกรองข้อมูลตามคำสำคัญ
print(filtered_by_user_request[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


Error: 'keywords' is not a dictionary.
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (8241, 16)
                      ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
0                   เสาหินบะซอลต์        บุรีรัมย์          14.583138   
1                อ่างเก็บน้ำลำพอก         สุรินทร์          14.931604   
2                อ่างเก็บน้ำลำพอก         สุรินทร์          14.931604   
3     กลุ่มสตรีทอผ้าบ้านท่ากระจาย     สุราษฎร์ธานี           9.590320   
4                  องค์ศรีสุขคเณศ         อุดรธานี          17.411384   
...                           ...              ...                ...   
8237                   วัดจอมธรรม        เชียงใหม่          18.632262   
8238                 น้ำตกวชิรธาร        เชียงใหม่          18.542003   
8239           น้ำพุร้อนโป่งเดือด        เชียงใหม่          19.243291   
8240                     ผาจูบกัน        เชียงใหม่          18.222642   
8241                       อ่างกา        เชียงใหม่          18.588454   

      LONGI

In [16]:
# ทดสอบกรองข้อมูลตามจังหวัดและประเภท
keywords = {
    'province': 'นครปฐม',
    'category': 'ธรรมชาติ'
}

# ตัวอย่าง df สมมุติ
df = pd.DataFrame({
    'ATT_NAME_TH': ['สถานที่1', 'สถานที่2', 'สถานที่3'],
    'PROVINCE_NAME_TH': ['นครปฐม', 'กรุงเทพฯ', 'นครปฐม'],
    'ATTR_CATAGORY_TH': ['แหล่งท่องเที่ยวทางธรรมชาติ', 'แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม', 'แหล่งท่องเที่ยวสำหรับกิจกรรมพิเศษ นันทนาการ และความสนใจพิเศษ'],
    'LATITUDE_LOCATION': [14.022788, 13.7563, 14.0234],
    'LONGITUDE_LOCATION': [99.978337, 100.5018, 99.9779]
})

# ทดสอบการกรองข้อมูล
filtered_df = filter_data(
    df=df,
    province=keywords['province'],
    category=keywords['category']
)

# แสดงผลลัพธ์
print(filtered_df[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'ATTR_CATAGORY_TH']])


Initial DataFrame shape: (3, 5)
Filtered DataFrame shape: (1, 5)
  ATT_NAME_TH PROVINCE_NAME_TH            ATTR_CATAGORY_TH
0    สถานที่1           นครปฐม  แหล่งท่องเที่ยวทางธรรมชาติ


In [9]:
# ฟังก์ชันการแยกคำสำคัญจากคำถามผู้ใช้
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# การตรวจสอบว่า keywords เป็น dictionary หรือไม่
if isinstance(keywords, dict):
    province = keywords.get('province')
    category = keywords.get('category')
else:
    # กรณีที่ keywords ไม่ใช่ dictionary ให้กำหนดค่าด้วยตัวแปรที่เหมาะสม
    province = None  # กำหนดค่า province เป็น None หรือค่าว่าง
    category = None   # กำหนดค่า category เป็น None หรือค่าว่าง
    # ถ้าต้องการให้แยกคำสำคัญจาก string สามารถเพิ่มการแยกคำได้ที่นี่

# กรองข้อมูลตามคำสำคัญของผู้ใช้
filtered_by_user_request = filter_data(
    df=df, 
    province=province,  # กรองจังหวัดตามคำสำคัญที่ได้
    category=category  # กรองประเภทตามคำสำคัญที่ได้
)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามพิกัดหลังจากกรองคำสั่งผู้ใช้แล้ว
filtered_by_location = filter_data(
    df=filtered_by_user_request,
    user_location=user_location,
    radius=radius
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:")
print(filtered_by_location[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- จังหวัด
- นครปฐม
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สวนสาธารณะ
- อุทยานแห่งชาติ
- แหล่งน้ำ
- ภูเขา

คำเหล่านี้สามารถใช้เพื่อค้นหาข้อมูลที่เกี่ยวข้องกับแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพมากขึ้น
Initial DataFrame shape: (8241, 15)
Filtered DataFrame shape: (8241, 15)
Initial DataFrame shape: (8241, 15)
Filtered DataFrame shape: (330, 16)
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
                       ATT_NAME_TH PROVINCE_NAME_TH  LATITUDE_LOCATION  \
48              ชุมชนเกาะลัดอีแท่น           นครปฐม          13.756728   
130    พิพิธภัณฑสถานแห่งชาติอู่ทอง       สุพรรณบุรี          14.372874   
165         วัดราษฎร์ศรัทธากะยาราม        สมุทรสาคร          13.590556   
191   กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย        กาญจนบุรี 

In [13]:
# ฟังก์ชันการแยกคำสำคัญจากคำถามผู้ใช้
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามพิกัดหลังจากกรองคำสั่งผู้ใช้แล้ว
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# กรองข้อมูลตามคำสำคัญของผู้ใช้
filtered_by_user_request = filter_data(
    df=filtered_by_location, 
    province = (keywords.get('province'),dict),
    category = (keywords.get('category'),dict)  # กรองประเภทตามคำสำคัญที่ได้
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัด
4. นครปฐม

คำสำคัญที่เกี่ยวข้องสำหรับการกรองข้อมูล:

- แหล่งท่องเที่ยว
- ธรรมชาติ
- นครปฐม
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สถานที่พักผ่อน
- ท่องเที่ยวเชิงนิเวศ

คำเหล่านี้สามารถใช้เพื่อค้นหาข้อมูลที่เกี่ยวข้องกับการท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพ!
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)


AttributeError: 'str' object has no attribute 'get'

In [ ]:
# แสดงค่าทั้งหมดใน keywords
print("ทั้งหมดใน keywords:", keywords)

# หรือดูเฉพาะกุญแจที่มีใน keywords
print("กุญแจทั้งหมดใน keywords:", keywords.keys())


ทั้งหมดใน keywords: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัด
4. นครปฐม

คำสำคัญที่เกี่ยวข้องสำหรับการกรองข้อมูล:

- แหล่งท่องเที่ยว
- ธรรมชาติ
- นครปฐม
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สถานที่พักผ่อน
- ท่องเที่ยวเชิงนิเวศ

คำเหล่านี้สามารถใช้เพื่อค้นหาข้อมูลที่เกี่ยวข้องกับการท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้อย่างมีประสิทธิภาพ!


AttributeError: 'str' object has no attribute 'keys'

# ลองแก้

In [ ]:
#3
def extract_keywords_from_query(query: str) -> str:
    """
    ใช้โมเดล LLM เพื่อแยกคำสำคัญจากคำถามของผู้ใช้
    """
    prompt = f"""
    กรุณาแยกคำสำคัญจากคำถามนี้: '{query}'
    คำสำคัญที่ต้องการให้แยกคือ:
    1. ชื่อจังหวัด (เช่น 'กรุงเทพฯ', 'เชียงใหม่', 'นครปฐม' เป็นต้น)
    2. ประเภทสถานที่ท่องเที่ยว (เช่น 'ธรรมชาติ', 'ทะเล', 'น้ำตก', 'ภูเขา', 'อุทยานแห่งชาติ' เป็นต้น)
    
    โปรดระบุจังหวัดและประเภทสถานที่จากคำถามนี้ และแสดงคำสำคัญที่สามารถใช้กรองข้อมูลได้
    """
    
    # ส่งคำถามไปยังโมเดล LLM
    response = llm.invoke(prompt)
    
    # ตรวจสอบผลลัพธ์และคืนค่าคำสำคัญ
    return response.content.strip()


In [18]:
# รับคำถามจากผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# การกรองข้อมูลตามคำสำคัญจากคำถามผู้ใช้
# การกรองข้อมูลจะทำใน filter_data โดยใช้คำสำคัญทั้งหมดที่ได้จาก keywords
# โดยที่ filter_data จะต้องทำการตรวจจับคำที่เกี่ยวข้องกับจังหวัดและประเภทเอง

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามระยะทางก่อน
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# กรองข้อมูลตามคำสำคัญทั้งหมดที่ได้จาก keywords
filtered_by_user_request = filter_data(
    df=filtered_by_location, 
    keywords=keywords  # ใช้คำสำคัญทั้งหมดที่ได้จากการแยกคำ
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'ATTR_CATAGORY_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' ได้แก่:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้:

- ท่องเที่ยว
- ธรรมชาติ
- จังหวัด
- นครปฐม
- สถานที่ท่องเที่ยว
- กิจกรรมกลางแจ้ง
- สถานที่ธรรมชาติ
- พื้นที่สีเขียว 

สามารถใช้คำสำคัญเหล่านี้ในการค้นหาข้อมูลหรือกรองข้อมูลเกี่ยวกับแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมได้ค่ะ
Initial DataFrame shape: (8241, 16)
Filtered DataFrame shape: (330, 16)


TypeError: filter_data() got an unexpected keyword argument 'keywords'

In [21]:
# ฟังก์ชัน filter_data ที่รองรับการกรองข้อมูลตามคำสำคัญ
def filter_data(df, user_location=None, radius=None, keywords=None):
    # ฟิลเตอร์ข้อมูลตามระยะทางหากมีการกำหนด
    if user_location and radius:
        # ทำการกรองตามระยะทาง (คุณอาจจะต้องเขียนฟังก์ชันคำนวณระยะทาง)
        df = filter_by_distance(df, user_location, radius)
    
    # ถ้ามีคำสำคัญให้กรองตามคำสำคัญเหล่านั้น
    if keywords:
        for keyword in keywords.split(','):  # สมมุติว่า keywords ถูกแยกด้วย comma
            keyword = keyword.strip()
            # กรองข้อมูลตามคำสำคัญที่เกี่ยวข้องกับจังหวัด
            if 'จังหวัด' in keyword:
                df = df[df['PROVINCE_NAME_TH'].str.contains(keyword, case=False, na=False)]
            # กรองข้อมูลตามคำสำคัญที่เกี่ยวข้องกับประเภท
            elif any(cat in keyword for cat in ['ธรรมชาติ', 'กิจกรรมกลางแจ้ง', 'อุทยานแห่งชาติ', 'ทะเล', 'ภูเขา', 'น้ำตก']):
                df = df[df['ATTR_CATAGORY_TH'].str.contains(keyword, case=False, na=False)]
    
    return df

# รับคำถามจากผู้ใช้
user_query = "ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย"

# ใช้ฟังก์ชันแยกคำสำคัญ
keywords = extract_keywords_from_query(user_query)
print("คำสำคัญที่ได้จากคำถาม:", keywords)

# ระบุพิกัดของผู้ใช้และรัศมี
user_location = (14.022788, 99.978337)  # พิกัดของผู้ใช้
radius = 50  # รัศมี 50 กิโลเมตร

# กรองข้อมูลตามระยะทางก่อน
filtered_by_location = filter_data(
    df=df,
    user_location=user_location,
    radius=radius
)

# กรองข้อมูลตามคำสำคัญทั้งหมดที่ได้จาก keywords
filtered_by_user_request = filter_data(
    df=filtered_by_location, 
    keywords=keywords  # ใช้คำสำคัญทั้งหมดที่ได้จากการแยกคำ
)

# แสดงผลลัพธ์สุดท้าย
print("สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:")
print(filtered_by_user_request[['ATT_NAME_TH', 'ATTR_CATAGORY_TH', 'PROVINCE_NAME_TH', 'LATITUDE_LOCATION', 'LONGITUDE_LOCATION']])


คำสำคัญที่ได้จากคำถาม: คำสำคัญจากคำถาม 'ช่วยหาแหล่งท่องเที่ยวทางธรรมชาติในจังหวัดนครปฐมให้หน่อย' สามารถแยกออกได้เป็น:

1. แหล่งท่องเที่ยว
2. ทางธรรมชาติ
3. จังหวัดนครปฐม

คำสำคัญที่เกี่ยวข้องที่สามารถใช้กรองข้อมูลได้ ได้แก่:

- ท่องเที่ยว
- ธรรมชาติ
- นครปฐม
- แหล่งท่องเที่ยวทางธรรมชาติ
- สถานที่ท่องเที่ยว
- จังหวัด
- ทัวร์ธรรมชาติ

คุณสามารถใช้คำเหล่านี้ในการค้นหาข้อมูลหรือกรองข้อมูลเกี่ยวกับแหล่งท่องเที่ยวในจังหวัดนครปฐมได้ค่ะ
สถานที่ท่องเที่ยวที่กรองแล้วในรัศมี 50 กิโลเมตร:
Empty DataFrame
Columns: [ATT_NAME_TH, ATTR_CATAGORY_TH, PROVINCE_NAME_TH, LATITUDE_LOCATION, LONGITUDE_LOCATION]
Index: []


In [20]:
import math

# ฟังก์ชันคำนวณระยะทางระหว่างสองพิกัด (ในหน่วยกิโลเมตร)
def haversine(lat1, lon1, lat2, lon2):
    # เปลี่ยนจากองศาเป็นเรเดียน
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # คำนวณความแตกต่างของละติจูดและลองจิจูด
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # รัศมีของโลก (ค่าเฉลี่ยประมาณ 6371 กิโลเมตร)
    radius = 6371.0
    
    # คำนวณระยะทาง
    distance = radius * c
    return distance

# ฟังก์ชันกรองข้อมูลตามระยะทาง
def filter_by_distance(df, user_location, radius):
    lat_user, lon_user = user_location
    df['distance'] = df.apply(lambda row: haversine(lat_user, lon_user, row['LATITUDE_LOCATION'], row['LONGITUDE_LOCATION']), axis=1)
    # กรองข้อมูลที่อยู่ในระยะทางที่กำหนด
    df = df[df['distance'] <= radius]
    return df


In [22]:
print(df['PROVINCE_NAME_TH'].unique())  # ตรวจสอบจังหวัดทั้งหมด
print(df['ATTR_CATAGORY_TH'].unique())  # ตรวจสอบประเภททั้งหมด


['บุรีรัมย์' 'สุรินทร์' 'สุราษฎร์ธานี' 'อุดรธานี' 'ชลบุรี' 'จันทบุรี'
 'ตรัง' 'สระบุรี' 'ระยอง' 'น่าน' 'ประจวบคีรีขันธ์' 'พระนครศรีอยุธยา'
 'เชียงใหม่' 'พิษณุโลก' 'เชียงราย' 'ลำปาง' 'ราชบุรี' 'สงขลา' 'ตาก' 'แพร่'
 'นครสวรรค์' 'อุบลราชธานี' 'สตูล' 'กระบี่' 'นครปฐม' 'ฉะเชิงเทรา'
 'นครราชสีมา' 'นนทบุรี' 'เพชรบุรี' 'พัทลุง' 'พิจิตร' 'สุพรรณบุรี'
 'ปทุมธานี' 'เลย' 'สมุทรปราการ' 'สมุทรสาคร' 'ลพบุรี' 'ศรีสะเกษ' 'อ่างทอง'
 'กรุงเทพมหานคร' 'หนองคาย' 'นครนายก' 'ขอนแก่น' 'ระนอง' 'กาฬสินธุ์'
 'สระแก้ว' 'ร้อยเอ็ด' 'กาญจนบุรี' 'แม่ฮ่องสอน' 'ปัตตานี' 'ปราจีนบุรี'
 'สกลนคร' 'เพชรบูรณ์' 'ลำพูน' 'นครศรีธรรมราช' 'ยะลา' 'ภูเก็ต' 'ชุมพร'
 'นครพนม' 'นราธิวาส' 'หนองบัวลำภู' 'พังงา' 'มุกดาหาร' 'ตราด' 'อุตรดิตถ์'
 'ชัยนาท' 'สุโขทัย' 'สิงห์บุรี' 'อำนาจเจริญ' 'พะเยา' 'บึงกาฬ' 'ยโสธร'
 'ชัยภูมิ' 'สมุทรสงคราม' 'อุทัยธานี' 'กำแพงเพชร' 'มหาสารคาม']
['แหล่งท่องเที่ยวทางธรรมชาติ'
 'แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม'
 'แหล่งท่องเที่ยวสำหรับกิจกรรมพิเศษ นันทนาการ และความสนใจพิเศษ']


In [23]:
print(filtered_by_location.shape[0])  # ตรวจสอบจำนวนข้อมูลหลังจากกรองตามระยะทาง


327


In [24]:
filtered_by_location = filter_data(df=df, user_location=user_location, radius=50)
print(filtered_by_location[['ATT_NAME_TH', 'ATTR_CATAGORY_TH', 'PROVINCE_NAME_TH']])

                           ATT_NAME_TH  \
48                  ชุมชนเกาะลัดอีแท่น   
130        พิพิธภัณฑสถานแห่งชาติอู่ทอง   
191       กลุ่มสตรีพัฒนากล้วยน้ำว้าไทย   
203                   วูดแลนด์เมืองไม้   
282   พิพิธภัณฑ์โรงเรียนวัดไร่ขิงวิทยา   
...                                ...   
8175                  สมาภรณ์ การ์เด้น   
8189                    สวนส้มโอไทยทวี   
8196                     วัดญาณเวศกวัน   
8197                      วัดดอนยายหอม   
8221                      บ้านสวนปรีดา   

                                       ATTR_CATAGORY_TH PROVINCE_NAME_TH  
48    แหล่งท่องเที่ยวสำหรับกิจกรรมพิเศษ นันทนาการ แล...           นครปฐม  
130         แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม       สุพรรณบุรี  
191         แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม        กาญจนบุรี  
203   แหล่งท่องเที่ยวสำหรับกิจกรรมพิเศษ นันทนาการ แล...           นครปฐม  
282         แหล่งท่องเที่ยวทางประวัติศาสตร์ และวัฒนธรรม           นครปฐม  
...                                          